In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE64738"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE64738"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE64738.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE64738.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE64738.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE64738_family.soft.gz', 'GSE64738_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE64738/GSE64738_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE64738/GSE64738_series_matrix.txt.gz


Background Information:
!Series_title	"Mithramycin Depletes Sp1 and Activates p53 to Mediate Senescence and Apoptosis of Pleural Mesothelioma Cells "
!Series_summary	"We used microarrays to detail the global gene expression of mithramycin treated cells, xenografts and sp1 depleted cells with p53 overexpression."
!Series_overall_design	"MPM cells and xenografts were treated with mithramycin or SP1 depleted/p53 overexpressed MPM cells were selected for RNA extraction and hybridization on Affymetrix microarrays. We sought to see the invitro and invivo effect of mithramycin (pharmacological model) and depletion of SP1 and overexpression of p53 (genetic model)."
Sample Characteristics Dictionary:
{0: ['cell type: Malignant pleural mesothelioma cells', 'cell type: Malignant pleural mesothelioma xenografts']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import re
import os
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on the series summary and overall design, this dataset appears to contain gene expression data
# from microarray experiments with Affymetrix microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Mesothelioma):
# From the sample characteristics dictionary, we can see at key 0, there's information about cell type
# indicating "Malignant pleural mesothelioma cells" and "Malignant pleural mesothelioma xenografts"
trait_row = 0  # The trait information is available at key 0

# For age:
# There is no information about the age of samples in the sample characteristics
age_row = None

# For gender:
# There is no information about the gender of samples in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (1 for Mesothelioma, 0 for control)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since all samples are mesothelioma (cells or xenografts), set all to 1
    if 'mesothelioma' in value.lower():
        return 1
    return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not used in this dataset as age information is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used in this dataset as gender information is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is from the previous step, likely extracted from the matrix file
    # and already available in memory. We'll use it directly.
    # If clinical_data is not defined, we should access the variable that contains the data
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use the data that should be available from previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    except NameError:
        print("Clinical data not available from previous step. Cannot proceed with clinical feature extraction.")


Preview of selected clinical data:
{'GSM1579060': [1.0], 'GSM1579061': [1.0], 'GSM1579062': [1.0], 'GSM1579063': [1.0], 'GSM1579064': [1.0], 'GSM1579065': [1.0], 'GSM1579066': [1.0], 'GSM1579067': [1.0], 'GSM1579068': [1.0], 'GSM1579069': [1.0], 'GSM1579070': [1.0], 'GSM1579071': [1.0], 'GSM1579072': [1.0], 'GSM1579073': [1.0], 'GSM1579074': [1.0], 'GSM1579075': [1.0], 'GSM1579076': [1.0], 'GSM1579077': [1.0], 'GSM1579078': [1.0], 'GSM1579079': [1.0], 'GSM1579080': [1.0], 'GSM1579081': [1.0], 'GSM1579082': [1.0], 'GSM1579083': [1.0], 'GSM1579084': [1.0], 'GSM1579085': [1.0], 'GSM1579086': [1.0], 'GSM1579087': [1.0], 'GSM1579088': [1.0], 'GSM1579089': [1.0], 'GSM1579090': [1.0], 'GSM1579091': [1.0], 'GSM1579092': [1.0], 'GSM1579093': [1.0], 'GSM1579094': [1.0], 'GSM1579095': [1.0], 'GSM1579096': [1.0], 'GSM1579097': [1.0], 'GSM1579098': [1.0], 'GSM1579099': [1.0], 'GSM1579100': [1.0], 'GSM1579101': [1.0], 'GSM1579102': [1.0], 'GSM1579103': [1.0], 'GSM1579104': [1.0]}
Clinical data saved

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on biomedical knowledge, the gene identifiers shown (like '1007_s_at', '1053_at', etc.)
# are Affymetrix probe set IDs (from the HG-U133 series platform), not human gene symbols.
# These IDs need to be mapped to standard human gene symbols for consistency and interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene identifiers and gene symbols from the annotation data
# Based on the preview, 'ID' contains the probe IDs (e.g., '1007_s_at') which match the gene expression data
# 'Gene Symbol' contains the human gene symbols we need (e.g., 'DDR1 /// MIR4640')
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
# This handles the many-to-many relations by splitting and summing appropriately
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print some statistics about the gene mapping process
print(f"Original number of probes: {len(gene_data.index)}")
print(f"Number of genes after mapping: {len(gene_data)}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Original number of probes: 21278
Number of genes after mapping: 21278
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Load the previously saved clinical data properly
try:
    # Try to load the previously saved clinical data with proper index
    selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Successfully loaded previously saved clinical data")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data index: {selected_clinical_df.index.tolist()}")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    print("Regenerating clinical data...")
    
    # Extract clinical data with correct trait_row value (0 from step 2)
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # Use correct row index from step 2
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("Before linking:")
print(f"Clinical data shape: {selected_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_data.shape}")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data after linking: {linked_data.shape}")
print(f"Columns in linked_data: {linked_data.columns.tolist()[:10]}...")  # Print first 10 columns

# 4. Handle missing values in the linked data
# Find the correct column name for the trait
trait_col = None
for col in linked_data.columns:
    if col == trait or 'Mesothelioma' in str(col):
        trait_col = col
        break

if trait_col is None and len(linked_data.columns) > 0:
    # If we can't find the trait column by name, use the first column (assuming it's the trait)
    trait_col = linked_data.columns[0]

print(f"Using trait column for missing value handling: {trait_col}")
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_col)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression from mesothelioma cells and xenografts. All samples have the same trait value (1.0), making the dataset biased for trait analysis."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Shape of normalized gene data: (19845, 45)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE64738.csv
Successfully loaded previously saved clinical data
Clinical data shape: (1, 44)
Clinical data index: [1.0]
Before linking:
Clinical data shape: (1, 44)
Genetic data shape: (19845, 45)
Shape of linked data after linking: (45, 19846)
Columns in linked_data: [1.0, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT']...
Using trait column for missing value handling: 1.0


Shape of linked data after handling missing values: (44, 19846)
Quartiles for '1.0':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature '1.0' in this dataset is severely biased.

Dataset validation failed. Final linked data not saved.
